The code in `pyldl.applications.emphasis_selection` is basically an unofficial implementation of the paper "Learning Emphasis Selection for Written Text in Visual Media from Crowd-Sourced Label Distributions" (*ACL* 2019).

In [1]:
from sklearn.model_selection import train_test_split
import pyldl.applications.emphasis_selection as es

In [2]:
words, freqs = es.load_semeval2020('./SemEval2020_Task10_Emphasis_Selection/')

In [3]:
es.visualization(words[:5])

In [4]:
words_train, words_test, freqs_train, freqs_test = \
train_test_split(words, freqs, test_size=0.2, random_state=0)

In [5]:
X_train, y_train, tokenizer, maxlen = es.preprocessing(words_train, freqs_train)

In [6]:
embeddings_matrix = es.load_glove('./glove.6B/', tokenizer, embedding_dim=100)

In [7]:
model = es.DL_BiLSTM(tokenizer, embeddings_matrix)

In [8]:
model.fit(X_train, y_train)

I0000 00:00:1709716004.999571   19425 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


In [9]:
X_test, y_test = es.preprocessing(words_test, freqs_test, tokenizer, maxlen)

In [10]:
y_pred = model.predict(X_test)

In [11]:
es.visualization(words_test[:5], y_pred[:5])

In [12]:
es.visualization(words_test[:5], y_test[:5])